In [356]:
from collections import defaultdict
import datetime

import string

import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.regression.quantile_regression import QuantReg

import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

plt.rcParams['figure.figsize'] = 10, 10
import seaborn as sns

%matplotlib inline

In [357]:
Iowa_Liqour_Sales = pd.read_csv('/Users/Snyder/Desktop/DSI-SF-1-SNYDER827/Projects/project-03/assets/Iowa_Liquor_Sales_reduced.csv')

print Iowa_Liqour_Sales.columns
Iowa_Liqour_Sales.head()

Index([u'Date', u'Store Number', u'City', u'Zip Code', u'County Number',
       u'County', u'Category', u'Category Name', u'Vendor Number',
       u'Item Number', u'Item Description', u'Bottle Volume (ml)',
       u'State Bottle Cost', u'State Bottle Retail', u'Bottles Sold',
       u'Sale (Dollars)', u'Volume Sold (Liters)', u'Volume Sold (Gallons)'],
      dtype='object')


,Date,Store Number,City,Zip Code,County Number,County,Category,Category Name,Vendor Number,Item Number,Item Description,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,03/31/2016,5029,DAVENPORT,52806,82.0,Scott,1022100.0,TEQUILA,370,87152,Avion Silver,375,$9.99,$14.99,12,$179.88,4.5,1.19
1,03/31/2016,5029,DAVENPORT,52806,82.0,Scott,1022100.0,TEQUILA,395,89197,Jose Cuervo Especial Reposado Tequila,1000,$12.50,$18.75,2,$37.50,2.0,0.53
2,03/31/2016,4959,CEDAR FALLS,50613,7.0,Black Hawk,1071100.0,AMERICAN COCKTAILS,380,63959,Uv Blue Raspberry Lemonade Pet,1750,$5.97,$8.96,6,$53.76,10.5,2.77
3,03/31/2016,2190,DES MOINES,50314,77.0,Polk,1031200.0,VODKA FLAVORED,205,40597,New Amsterdam Red Berry,200,$2.24,$3.36,48,$161.28,9.6,2.54
4,03/31/2016,5240,WEST BRANCH,52358,NaN,NaN,1081200.0,CREAM LIQUEURS,305,73055,Rumchata,750,$12.50,$18.75,6,$112.50,4.5,1.19


## Clean the data

Let's practice our data cleaning skills on the Project 3 dataset. If you don't remember how to do any of these tasks, look back at your work from the previous weeks or search the internet. There are many blog articles and Stack Overflow posts that cover these topics.

You'll want to complete at least the following tasks:
* Remove redundant columns
* Remove "$" prices from characters and convert values to floats.
* Convert dates to pandas datetime objects
* Convert category floats to integers
* Drop or fill in bad values

In [159]:
print Iowa_Liqour_Sales.shape
Iowa_Liqour_Sales.info()

(2709552, 18)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2709552 entries, 0 to 2709551
Data columns (total 18 columns):
Date                   object
Store_Number           int64
City                   object
Zip_Code               object
County_Number          float64
County                 object
Category               float64
Category_Name          object
Vendor_Number          int64
Item_Number            int64
Item_Description       object
Bottle_Volume_ml       int64
State_Bottle_Cost      object
State_Bottle_Retail    object
Bottles_Sold           int64
Sale_Dollars           object
Volume_Sold_Liters     float64
Volume_Sold_Gallons    float64
dtypes: float64(4), int64(5), object(9)
memory usage: 372.1+ MB


In [358]:
#Replace spaces (" ") with underscores ("_") 
Iowa_Liqour_Sales.columns = Iowa_Liqour_Sales.columns.map(lambda x: x.replace(' ', '_'))

Iowa_Liqour_Sales.columns

Index([u'Date', u'Store_Number', u'City', u'Zip_Code', u'County_Number',
       u'County', u'Category', u'Category_Name', u'Vendor_Number',
       u'Item_Number', u'Item_Description', u'Bottle_Volume_(ml)',
       u'State_Bottle_Cost', u'State_Bottle_Retail', u'Bottles_Sold',
       u'Sale_(Dollars)', u'Volume_Sold_(Liters)', u'Volume_Sold_(Gallons)'],
      dtype='object')

In [137]:
len(Iowa_Liqour_Sales.City.unique())

386

In [359]:
#Convert Zip_Code to string
Iowa_Liqour_Sales.Zip_Code = Iowa_Liqour_Sales.Zip_Code.map(str)

In [360]:
#Zip Code for Dunlap is 51529. Originally in there as '712-2'
Iowa_Liqour_Sales.Zip_Code = Iowa_Liqour_Sales.Zip_Code.map(lambda x : x.replace('712-2', '51529'))

In [361]:
#Convert Zip_Code back to int
Iowa_Liqour_Sales.Zip_Code = Iowa_Liqour_Sales.Zip_Code.map(int)

In [350]:
Iowa_Liqour_Sales.Zip_Code.unique()

Iowa_Liqour_Sales[Iowa_Liqour_Sales.Zip_Code == 51529]
#Zip Code for Dunlap is 51529. Originally in there as '712-2

,Date,Store_Number,City,Zip_Code,County_Number,County,Category,Category_Name,Vendor_Number,Item_Number,Item_Description,Bottle_Volume_ml,State_Bottle_Cost,State_Bottle_Retail,Bottles_Sold,Sale_Dollars,Volume_Sold_Liters,Volume_Sold_Gallons
28205,03/28/2016,4307,DUNLAP,51529,NaN,NaN,1012100.0,CANADIAN WHISKIES,434,13037,Canadian Reserve Whisky,1000,$5.31,$7.97,4,$31.88,4.00,1.06
28297,03/28/2016,4307,DUNLAP,51529,NaN,NaN,1031200.0,VODKA FLAVORED,380,41601,Uv Blue (raspberry) Vodka,375,$3.45,$5.18,4,$20.72,1.50,0.40
28362,03/28/2016,4307,DUNLAP,51529,NaN,NaN,1081200.0,CREAM LIQUEURS,260,68022,Bailey's Salted Caramel,750,$12.50,$18.75,1,$18.75,0.75,0.20
28366,03/28/2016,4307,DUNLAP,51529,NaN,NaN,1022100.0,TEQUILA,410,88296,Patron Tequila Silver,750,$27.00,$40.50,2,$81.00,1.50,0.40
28387,03/28/2016,4307,DUNLAP,51529,NaN,NaN,1022100.0,TEQUILA,85,3657,Herradura Gold Reposado 6pak,750,$23.58,$35.37,1,$35.37,0.75,0.20
28407,03/28/2016,4307,DUNLAP,51529,NaN,NaN,1081010.0,AMERICAN AMARETTO,55,73986,Di Amore Amaretto,750,$6.00,$9.00,4,$36.00,3.00,0.79
28408,03/28/2016,4307,DUNLAP,51529,NaN,NaN,1031080.0,VODKA 80 PROOF,260,36186,Gordon's Vodka 80 Prf,750,$5.26,$7.89,4,$31.56,3.00,0.79
28409,03/28/2016,4307,DUNLAP,51529,NaN,NaN,1062300.0,FLAVORED RUM,65,44520,Cruzan Vanilla Rum,750,$6.83,$10.25,1,$10.25,0.75,0.20
28410,03/28/2016,4307,DUNLAP,51529,NaN,NaN,1032080.0,IMPORTED VODKA,370,34007,Absolut Swedish Vodka 80 Prf,1000,$14.99,$22.49,12,$269.88,12.00,3.17
28411,03/28/2016,4307,DUNLAP,51529,NaN,NaN,1031080.0,VODKA 80 PROOF,380,37348,Phillips Vodka,1750,$7.60,$11.40,6,$68.40,10.50,2.77


In [139]:
len(Iowa_Liqour_Sales.County.unique())

100

In [362]:
#Replace parentheses with blanks
Iowa_Liqour_Sales.columns = Iowa_Liqour_Sales.columns.map(lambda x: x.replace('(', '').replace(')', ''))

Iowa_Liqour_Sales.head(0)

,Date,Store_Number,City,Zip_Code,County_Number,County,Category,Category_Name,Vendor_Number,Item_Number,Item_Description,Bottle_Volume_ml,State_Bottle_Cost,State_Bottle_Retail,Bottles_Sold,Sale_Dollars,Volume_Sold_Liters,Volume_Sold_Gallons


In [352]:
#Convert cities and counties to upper case
Iowa_Liqour_Sales.City = Iowa_Liqour_Sales.City.map(string.upper)
#Iowa_Liqour_Sales.County = Iowa_Liqour_Sales.County.map(string.upper)

In [152]:
Iowa_Liqour_Sales.County.unique()

array(['Scott', 'Black Hawk', 'Polk', nan, 'Cerro Gordo', 'Johnson',
       'Greene', 'Worth', 'Poweshiek', 'Crawford', 'Winnebago', 'Boone',
       'Clinton', 'Linn', 'Clarke', 'Cedar', 'Decatur', 'Warren', 'Jasper',
       'Iowa', 'Madison', 'Carroll', 'Dallas', 'Story', 'Buena Vista',
       'Woodbury', 'Appanoose', 'Wapello', 'Winneshiek', 'Fayette',
       'Marion', 'Calhoun', 'Monroe', 'Monona', 'Mahaska', 'Webster',
       'Allamakee', 'Buchanan', 'Bremer', 'Hancock', 'Kossuth', 'Sac',
       'Plymouth', 'Ida', 'Lucas', 'Davis', 'Benton', 'Clay', 'Mitchell',
       'Dubuque', 'Pottawattamie', 'Floyd', 'Hamilton', 'Sioux',
       'Marshall', 'Union', "O'Brien", 'Tama', 'Montgomery', 'Taylor',
       'Humboldt', 'Dickinson', 'Lyon', 'Jones', 'Wright', 'Cherokee',
       'Butler', 'Page', 'Emmet', 'Palo Alto', 'Adair', 'Mills',
       'Ringgold', 'Adams', 'Howard', 'Chickasaw', 'Hardin', 'Grundy',
       'Pocahontas', 'Osceola', 'Clayton', 'Jefferson', 'Muscatine',
       'Des Moin

In [210]:
Iowa_Liqour_Sales[Iowa_Liqour_Sales.Zip_Code == 50601]

,Date,Store_Number,City,Zip_Code,County_Number,County,Category,Category_Name,Vendor_Number,Item_Number,Item_Description,Bottle_Volume_ml,State_Bottle_Cost,State_Bottle_Retail,Bottles_Sold,Sale_Dollars,Volume_Sold_Liters,Volume_Sold_Gallons
35312,03/28/2016,3591,ACKLEY,50601,NaN,NaN,1012100.0,CANADIAN WHISKIES,55,12408,Canadian Ltd Whisky,1750,$9.14,$13.71,6,$82.26,10.50,2.77
35466,03/28/2016,3591,ACKLEY,50601,NaN,NaN,1031080.0,VODKA 80 PROOF,434,36308,Hawkeye Vodka,1750,$7.17,$10.76,6,$64.56,10.50,2.77
38609,03/28/2016,3591,ACKLEY,50601,NaN,NaN,1012100.0,CANADIAN WHISKIES,115,10550,Black Velvet Toasted Caramel,750,$6.75,$10.13,12,$121.56,9.00,2.38
38611,03/28/2016,3591,ACKLEY,50601,NaN,NaN,1081600.0,WHISKEY LIQUEUR,421,86887,Southern Comfort,1000,$12.97,$19.46,12,$233.52,12.00,3.17
38615,03/28/2016,3591,ACKLEY,50601,NaN,NaN,1062310.0,SPICED RUM,260,43337,Captain Morgan Spiced Rum,1000,$11.75,$17.63,12,$211.56,12.00,3.17
38616,03/28/2016,3591,ACKLEY,50601,NaN,NaN,1012300.0,IRISH WHISKIES,370,15627,Jameson,1000,$18.49,$27.74,12,$332.88,12.00,3.17
38617,03/28/2016,3591,ACKLEY,50601,NaN,NaN,1012100.0,CANADIAN WHISKIES,115,11777,Black Velvet,1000,$6.63,$9.95,12,$119.40,12.00,3.17
69894,03/21/2016,4415,ACKLEY,50601,42.0,Hardin,1062310.0,SPICED RUM,260,43331,Captain Morgan Spiced Rum Mini,500,$4.80,$7.20,2,$14.40,1.00,0.26
69980,03/21/2016,4415,ACKLEY,50601,42.0,Hardin,1051010.0,AMERICAN GRAPE BRANDIES,205,52596,E & J Vs,750,$6.00,$9.00,2,$18.00,1.50,0.40
69993,03/21/2016,4415,ACKLEY,50601,42.0,Hardin,1062310.0,SPICED RUM,260,43334,Captain Morgan Spiced Rum,375,$5.00,$7.50,4,$30.00,1.50,0.40


In [186]:
#Count number of null values
#Iowa_Liqour_Sales.isnull().sum()
Iowa_Liqour_Sales[Iowa_Liqour_Sales.County.isnull()].Category_Name.unique()

array(['CREAM LIQUEURS', 'TENNESSEE WHISKIES', 'TRIPLE SEC',
       'WHISKEY LIQUEUR', 'PEPPERMINT SCHNAPPS', 'TEQUILA', 'SPICED RUM',
       'AMERICAN GRAPE BRANDIES', 'PUERTO RICO & VIRGIN ISLANDS RUM',
       'BLENDED WHISKIES', 'PEACH BRANDIES',
       'MISC. IMPORTED CORDIALS & LIQUEURS', 'STRAIGHT BOURBON WHISKIES',
       'IRISH WHISKIES', 'IMPORTED VODKA - MISC', 'VODKA 80 PROOF',
       'BLACKBERRY BRANDIES', 'CANADIAN WHISKIES', 'IMPORTED VODKA',
       'APRICOT BRANDIES', 'AMERICAN COCKTAILS', 'SCOTCH WHISKIES',
       'GRAPE SCHNAPPS', 'IMPORTED SCHNAPPS', 'PEACH SCHNAPPS',
       'FLAVORED RUM', 'VODKA FLAVORED', 'IMPORTED GRAPE BRANDIES',
       'IMPORTED DRY GINS', 'AMERICAN DRY GINS', 'BUTTERSCOTCH SCHNAPPS',
       'COFFEE LIQUEURS', '100 PROOF VODKA', 'WATERMELON SCHNAPPS',
       'CINNAMON SCHNAPPS', 'APPLE SCHNAPPS',
       'MISC. AMERICAN CORDIALS & LIQUEURS', 'AMERICAN AMARETTO',
       'MISCELLANEOUS  BRANDIES', 'MISCELLANEOUS SCHNAPPS',
       'SPEARMINT SCHNAPP

In [413]:
#Create lookup table with no null values
Lookup_ILS_non_null = Iowa_Liqour_Sales.dropna(axis=0)

print Lookup_ILS_non_null.shape

Lookup_ILS_non_null.isnull().sum()

(2692602, 18)


Date                   0
Store_Number           0
City                   0
Zip_Code               0
County_Number          0
County                 0
Category               0
Category_Name          0
Vendor_Number          0
Item_Number            0
Item_Description       0
Bottle_Volume_ml       0
State_Bottle_Cost      0
State_Bottle_Retail    0
Bottles_Sold           0
Sale_Dollars           0
Volume_Sold_Liters     0
Volume_Sold_Gallons    0
dtype: int64

In [415]:
#Drop counties for merging later
Drop_county_ILS = Iowa_Liqour_Sales.drop('County', axis = 1)
Drop_county_ILS = Drop_county_ILS.drop('County_Number', axis = 1)
Drop_county_ILS.isnull().sum()

#Will fix null categories in another iteration

Date                      0
Store_Number              0
City                      0
Zip_Code                  0
Category                779
Category_Name          6109
Vendor_Number             0
Item_Number               0
Item_Description          0
Bottle_Volume_ml          0
State_Bottle_Cost         0
State_Bottle_Retail       0
Bottles_Sold              0
Sale_Dollars              0
Volume_Sold_Liters        0
Volume_Sold_Gallons       0
dtype: int64

In [213]:
Lookup_County.Zip_Code.value_counts()

50248    2
50702    2
52577    2
50247    2
50266    2
50325    2
52722    2
51632    2
50266    2
50501    2
52405    2
50129    2
50129    2
50601    2
52317    2
50248    2
50325    2
50501    2
50702    2
52722    2
52577    2
50220    2
50220    2
51632    2
50511    1
52501    1
52248    1
50219    1
52052    1
50002    1
        ..
52655    1
52656    1
50588    1
50171    1
50170    1
50619    1
51453    1
51455    1
50846    1
52553    1
51442    1
52627    1
56201    1
52732    1
50801    1
52254    1
50131    1
52342    1
52154    1
50020    1
51640    1
51106    1
51027    1
51450    1
50854    1
50475    1
50851    1
52214    1
50579    1
52224    1
Name: Zip_Code, dtype: int64

In [416]:
#Create series of lookup tables for counties to merge.
Lookup_County = pd.DataFrame()
Lookup_County['City'] = Lookup_ILS_non_null['City']
Lookup_County['County'] = Lookup_ILS_non_null['County']
Lookup_County['County_Number'] = Lookup_ILS_non_null['County_Number']
#print Lookup_County.shape
#Lookup_County.sort('Zip_Code')

keys = ['City', 'County', 'County_Number']
grouped = Lookup_County.groupby(keys)
index = [gp_keys[0] for gp_keys in grouped.groups.values()]
Lookup_County = Lookup_County.reindex(index)

Lookup_County.sort('City')

# #Merge Lookup_County DF with DF without counties. Will have to do this twice as some cities occupy multipl counties
# County_merge_1 = pd.merge(Drop_county_ILS, Lookup_County, how='left', on='Zip_Code')
# #County_merge_1.isnull().sum() #178 null counties to go.

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:14: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,City,County,County_Number
69894,ACKLEY,Hardin,42.0
31550,ADAIR,Adair,1.0
28450,ADEL,Dallas,25.0
63057,AFTON,Union,88.0
62036,AKRON,Plymouth,75.0
7257,ALBIA,Monroe,68.0
20175,ALDEN,Hardin,42.0
7796,ALGONA,Kossuth,55.0
21648,ALLISON,Butler,12.0
385057,ALTA,Buena Vista,11.0


In [297]:
#Seferal Cities were mislabeld with two counties while only Gilmore, Sheldahl, and West Des Moines occupy multiple counties.
Iowa_Liqour_Sales[Iowa_Liqour_Sales.City == u'NEWTON'].loc[:, 'County'] = Iowa_Liqour_Sales[Iowa_Liqour_Sales.City == u'NEWTON']['County'].map(lambda x : x.replace('Webster', 'Jasper'))

In [255]:
Iowa_Liquor_Sales = Iowa_Liqour_Sales[Iowa_Liqour_Sales.City == u'NEWTON'].County.map(lambda x : x.replace('Webster', 'Jasper'))

In [298]:
# Iowa_Liqour_Sales[Iowa_Liqour_Sales.City == 'NEWTON']['County'].value_counts()
# Iowa_Liqour_Sales[Iowa_Liqour_Sales.City == 'NEWTON'].loc[:, 'County']

# Iowa_Liqour_Sales[Iowa_Liqour_Sales.City == 'NEWTON']['County'].value_counts()
Iowa_Liqour_Sales[Iowa_Liqour_Sales.City == 'NEWTON']['County'].value_counts().index

Index([u'Jasper', u'Webster'], dtype='object')

In [364]:
#Replace Webster County with 'Jasper' for Newton City

def replace_county(row):

    if row['County'] == "Webster":
        row['County'] = "Jasper"
        row['County_Number'] = 50.0
    return row

newton_mask      =  Iowa_Liqour_Sales.City == u'NEWTON'
webster_mask     =  Iowa_Liqour_Sales['County'] == 'Webster'
Iowa_Liqour_Sales[(newton_mask) & (webster_mask)] = Iowa_Liqour_Sales[(newton_mask) & (webster_mask)].apply(replace_county, axis=1)
# update_indexes   =  Iowa_Liqour_Sales[(newton_mask) & (webster_mask)]['County'].index
# Iowa_Liqour_Sales.iloc[update_indexes]["County"] = new_counties
# new_counties.value_counts()

In [368]:
#Replace Webster County with 'Hardin' for City of Ackley

def replace_county(row):

    if row['County'] == "Webster":
        row['County'] = "Hardin"
        row['County_Number'] = 42.0
    return row

city_mask      =  Iowa_Liqour_Sales.City == u'ACKLEY'
county_mask     =  Iowa_Liqour_Sales['County'] == 'Webster'
Iowa_Liqour_Sales[(city_mask) & (county_mask)] = Iowa_Liqour_Sales[(city_mask) & (county_mask)].apply(replace_county, axis=1)

In [365]:
Iowa_Liqour_Sales[Iowa_Liqour_Sales.City == 'NEWTON'].County.value_counts()

Jasper    25597
Name: County, dtype: int64

In [369]:
Iowa_Liqour_Sales[Iowa_Liqour_Sales.City == 'ACKLEY'].County.value_counts()

Hardin    885
Name: County, dtype: int64

In [375]:
#North Liberty is in Johnson County, not Iowa County
Iowa_Liqour_Sales[Iowa_Liqour_Sales.City == 'NORTH LIBERTY'].County.value_counts()

Johnson    13084
Name: County, dtype: int64

In [374]:
#Replace Iowa County with 'Johnson' for City of 'North Liberty

def replace_county(row):

    if row['County'] == "Iowa": #Original county
        row['County'] = "Johnson" #New county
        row['County_Number'] = 52.0 #Corresponding County Number for new county
    return row

city_mask      =  Iowa_Liqour_Sales.City == u'NORTH LIBERTY'
county_mask     =  Iowa_Liqour_Sales['County'] == 'Iowa'
Iowa_Liqour_Sales[(city_mask) & (county_mask)] = Iowa_Liqour_Sales[(city_mask) & (county_mask)].apply(replace_county, axis=1)

In [384]:
#Bettendorf is in 'Scott' County, not Iowa County
Iowa_Liqour_Sales[Iowa_Liqour_Sales.City == 'BETTENDORF'].County.value_counts()
Iowa_Liqour_Sales[Iowa_Liqour_Sales.City == 'BETTENDORF'].County_Number.value_counts()

82.0    36786
Name: County_Number, dtype: int64

In [383]:
#Replace Iowa County with 'Scott' for City of 'Bettendorf'

def replace_county(row):

    if row['County'] == "Iowa": #Original county
        row['County'] = "Scott" #New county
        row['County_Number'] = 82.0 #Corresponding County Number for new county
    return row

city_mask      =  Iowa_Liqour_Sales.City == u'BETTENDORF'
county_mask     =  Iowa_Liqour_Sales['County'] == 'Iowa'
Iowa_Liqour_Sales[(city_mask) & (county_mask)] = Iowa_Liqour_Sales[(city_mask) & (county_mask)].apply(replace_county, axis=1)

In [389]:
#Replace 'Clayton' County with 'Adams' for City of 'Corning'

def replace_county(row):

    if row['County'] == "Clayton": #Original county
        row['County'] = "Adams" #New county
        row['County_Number'] = 2.0 #Corresponding County Number for new county
    return row

city_mask      =  Iowa_Liqour_Sales.City == u'CORNING'
county_mask     =  Iowa_Liqour_Sales['County'] == 'Clayton'
Iowa_Liqour_Sales[(city_mask) & (county_mask)] = Iowa_Liqour_Sales[(city_mask) & (county_mask)].apply(replace_county, axis=1)

In [393]:
#Replace 'Hardin' County with 'Webster' for City of 'FORT DODGE'

def replace_county(row):

    if row['County'] == "Hardin": #Original county
        row['County'] = "Webster" #New county
        row['County_Number'] = 94.0 #Corresponding County Number for new county
    return row

city_mask      =  Iowa_Liqour_Sales.City == u'FORT DODGE'
county_mask     =  Iowa_Liqour_Sales['County'] == 'Hardin' #Original or wrong county
Iowa_Liqour_Sales[(city_mask) & (county_mask)] = Iowa_Liqour_Sales[(city_mask) & (county_mask)].apply(replace_county, axis=1)

In [397]:
#Replace 'Boone' County with 'Dallas' for City of 'PERRY'

def replace_county(row):

    if row['County'] == "Boone": #Original county
        row['County'] = "Dallas" #New county
        row['County_Number'] = 25.0 #Corresponding County Number for new county
    return row

city_mask      =  Iowa_Liqour_Sales.City == u'PERRY'
county_mask     =  Iowa_Liqour_Sales['County'] == 'Boone'
Iowa_Liqour_Sales[(city_mask) & (county_mask)] = Iowa_Liqour_Sales[(city_mask) & (county_mask)].apply(replace_county, axis=1)

In [ ]:
#Replace 'Boone' County with 'Dallas' for City of 'PERRY'

def replace_county(row):

    if row['County'] == "Boone": #Original county
        row['County'] = "Dallas" #New county
        row['County_Number'] = 25.0 #Corresponding County Number for new county
    return row

city_mask      =  Iowa_Liqour_Sales.City == u'PERRY'
county_mask     =  Iowa_Liqour_Sales['County'] == 'Boone'
Iowa_Liqour_Sales[(city_mask) & (county_mask)] = Iowa_Liqour_Sales[(city_mask) & (county_mask)].apply(replace_county, axis=1)

In [406]:
#Replace 'Buchanan' County with 'Mahaska' for City of 'OSKALOOSA'

def replace_county(row):

    if row['County'] == "Buchanan": #Original county
        row['County'] = "Mahaska" #New county
        row['County_Number'] = 62.0 #Corresponding County Number for new county
    return row

city_mask      =  Iowa_Liqour_Sales.City == u'OSKALOOSA'
county_mask     =  Iowa_Liqour_Sales['County'] == 'Buchanan'
Iowa_Liqour_Sales[(city_mask) & (county_mask)] = Iowa_Liqour_Sales[(city_mask) & (county_mask)].apply(replace_county, axis=1)

In [411]:
#Replace 'Iowa' County with 'Marshall' for City of 'STATE CENTER'

def replace_county(row):

    if row['County'] == "Iowa": #Original county
        row['County'] = "Marshall" #New county
        row['County_Number'] = 64.0 #Corresponding County Number for new county
    return row

city_mask      =  Iowa_Liqour_Sales.City == u'STATE CENTER'
county_mask     =  Iowa_Liqour_Sales['County'] == 'Iowa'
Iowa_Liqour_Sales[(city_mask) & (county_mask)] = Iowa_Liqour_Sales[(city_mask) & (county_mask)].apply(replace_county, axis=1)

In [409]:
Lookup_County[Lookup_County.City == 'STATE CENTER']

,City,County,County_Number
18221,STATE CENTER,Marshall,64.0
18333,STATE CENTER,Iowa,48.0


In [412]:
Iowa_Liqour_Sales[Iowa_Liqour_Sales.City == 'STATE CENTER'].County.value_counts()
#Iowa_Liqour_Sales[Iowa_Liqour_Sales.City == 'OSKALOOSA']

Marshall    1814
Name: County, dtype: int64

In [228]:
Lookup_County.City.value_counts()

NEWTON             2
ACKLEY             2
NORTH LIBERTY      2
BETTENDORF         2
CORNING            2
FORT DODGE         2
PERRY              2
CLIVE              2
WEST DES MOINES    2
OSKALOOSA          2
STATE CENTER       2
MISSOURI VALLEY    1
COLFAX             1
RAYMOND            1
STORY CITY         1
GOWRIE             1
ADAIR              1
STORM LAKE         1
WAVERLY            1
JESUP              1
DENVER             1
CORALVILLE         1
GREENE             1
LE CLAIRE          1
HARLAN             1
BELLEVUE           1
IOWA FALLS         1
ANKENY             1
ARMSTRONG          1
ANTHON             1
                  ..
BURLINGTON         1
GRIMES             1
NEW SHARON         1
MALVERN            1
SUMNER             1
TRAER              1
OKOBOJI            1
MARENGO            1
VINTON             1
SPENCER            1
MASON CITY         1
STUART             1
ATLANTIC           1
PANORA             1
KNOXVILLE          1
NORTH ENGLISH      1
BLOOMFIELD   

In [216]:
County_merge_1.isnull().sum()

Date                      0
Store_Number              0
City                      0
Zip_Code                  0
Category                894
Category_Name          6712
Vendor_Number             0
Item_Number               0
Item_Description          0
Bottle_Volume_ml          0
State_Bottle_Cost         0
State_Bottle_Retail       0
Bottles_Sold              0
Sale_Dollars              0
Volume_Sold_Liters        0
Volume_Sold_Gallons       0
County                  178
County_Number           178
dtype: int64

In [ ]:
df_copy = Iowa_Liqour_Sales.copy()

df_copy.interpolate(method = 'linear', inplace = True, limit_direction = 'both')
for group_label, group_df in df_copy.groupby("Store Number"):
    df_copy

In [85]:
dfc = Iowa_Liqour_Sales.copy()
# dfc.interpolate(method = 'linear', inplace = True, limit_direction = 'both')
# for group_label, group_df in dfc.groupby("City"):
#     dfc[dfc["City"] == group_label] = dfc[dfc["City"] == group_label].sort("County_Number").ffill()
#     dfc[dfc["City"] == group_label] = dfc[dfc["City"] == group_label].sort("County").ffill()
    
# dfc.info()

for group_label, group_df in dfc.groupby(['Store_Number','City']):
    store_mask = dfc['Store_Number'] == group_label[0]
    city_mask = dfc['City'] == group_label[1]
    dfc[(store_mask) & (city_mask)] = dfc[(store_mask) & (city_mask)].sort('County').ffill()
    
    

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:12: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


KeyboardInterrupt: 

In [124]:
df_head = Iowa_Liqour_Sales.head(1000)

#for group_label, group_df in df_head.groupby(['City']):
for group_label, group_df in df_head.groupby(['City','Zip_Code']):
    city_mask = df_head['City'] == group_label[0]
    zip_mask = df_head['Zip_Code'] == group_label[1]
    df_head[(city_mask) & (zip_mask)] = df_head[(city_mask) & (zip_mask)].sort('County').ffill()
#     df_head[(city_mask)] = df_head[(city_mask)].sort('County').ffill()
df_head.info()

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
Date                     1000 non-null object
Store_Number             1000 non-null int64
City                     1000 non-null object
Zip_Code                 1000 non-null object
County_Number            962 non-null float64
County                   962 non-null object
Category                 1000 non-null float64
Category_Name            1000 non-null object
Vendor_Number            1000 non-null int64
Item_Number              1000 non-null int64
Item_Description         1000 non-null object
Bottle_Volume_(ml)       1000 non-null int64
State_Bottle_Cost        1000 non-null object
State_Bottle_Retail      1000 non-null object
Bottles_Sold             1000 non-null int64
Sale_(Dollars)           1000 non-null object
Volume_Sold_(Liters)     1000 non-null float64
Volume_Sold_(Gallons)    1000 non-null float64
dtypes: float64(4), int64(5), object(9)
memory usage: 140.7+ KB


In [109]:
len(df_head[df_head['County'].isnull()]['Store_Number'].unique())

1

In [128]:
df_head = Iowa_Liqour_Sales.head(1000)

#for group_label, group_df in df_head.groupby(['City']):
for group_label, group_df in df_head.groupby(['City','Zip_Code']):
    city_mask = df_head['City'] == group_label[0]
    zip_mask = df_head['Zip_Code'] == group_label[1]
    
print group_label[0]

WEST DES MOINES


In [130]:
df_head

,Date,Store_Number,City,Zip_Code,County_Number,County,Category,Category_Name,Vendor_Number,Item_Number,Item_Description,Bottle_Volume_(ml),State_Bottle_Cost,State_Bottle_Retail,Bottles_Sold,Sale_(Dollars),Volume_Sold_(Liters),Volume_Sold_(Gallons)
0,03/31/2016,5029,DAVENPORT,52806,82.0,Scott,1022100.0,TEQUILA,370,87152,Avion Silver,375,$9.99,$14.99,12,$179.88,4.5,1.19
1,03/31/2016,5029,DAVENPORT,52806,82.0,Scott,1022100.0,TEQUILA,395,89197,Jose Cuervo Especial Reposado Tequila,1000,$12.50,$18.75,2,$37.50,2.0,0.53
2,03/31/2016,4959,CEDAR FALLS,50613,7.0,Black Hawk,1071100.0,AMERICAN COCKTAILS,380,63959,Uv Blue Raspberry Lemonade Pet,1750,$5.97,$8.96,6,$53.76,10.5,2.77
3,03/31/2016,2190,DES MOINES,50314,77.0,Polk,1031200.0,VODKA FLAVORED,205,40597,New Amsterdam Red Berry,200,$2.24,$3.36,48,$161.28,9.6,2.54
4,03/31/2016,5240,WEST BRANCH,52358,NaN,NaN,1081200.0,CREAM LIQUEURS,305,73055,Rumchata,750,$12.50,$18.75,6,$112.50,4.5,1.19


In [123]:
Iowa_Liqour_Sales[(Iowa_Liqour_Sales['City'] == 'WEST BRANCH') & (Iowa_Liqour_Sales['Zip_Code'] == 52358)]

,Date,Store_Number,City,Zip_Code,County_Number,County,Category,Category_Name,Vendor_Number,Item_Number,Item_Description,Bottle_Volume_(ml),State_Bottle_Cost,State_Bottle_Retail,Bottles_Sold,Sale_(Dollars),Volume_Sold_(Liters),Volume_Sold_(Gallons)
366012,01/28/2016,4170,WEST BRANCH,52358,16.0,Cedar,1031080.0,VODKA 80 PROOF,434,36306,Hawkeye Vodka,750,$3.34,$5.01,2,$10.02,1.50,0.40
366159,01/28/2016,4170,WEST BRANCH,52358,16.0,Cedar,1062310.0,SPICED RUM,240,45887,Sailor Jerry Spiced Navy Rum,1000,$11.78,$17.67,1,$17.67,1.00,0.26
366282,01/28/2016,4170,WEST BRANCH,52358,16.0,Cedar,1062200.0,PUERTO RICO & VIRGIN ISLANDS RUM,35,43124,Bacardi Superior Rum,375,$4.50,$6.75,2,$13.50,0.75,0.20
366294,01/28/2016,4170,WEST BRANCH,52358,16.0,Cedar,1042100.0,IMPORTED DRY GINS,260,28866,Tanqueray Gin,750,$12.50,$18.75,2,$37.50,1.50,0.40
366313,01/28/2016,4170,WEST BRANCH,52358,16.0,Cedar,1062200.0,PUERTO RICO & VIRGIN ISLANDS RUM,434,46350,Hawkeye Light Rum,1750,$7.67,$11.51,6,$69.06,10.50,2.77
366490,01/28/2016,4170,WEST BRANCH,52358,16.0,Cedar,1081030.0,COFFEE LIQUEURS,370,67526,Kahlua Coffee Liqueur,750,$11.99,$17.99,1,$17.99,0.75,0.20
366601,01/28/2016,4170,WEST BRANCH,52358,16.0,Cedar,1011100.0,BLENDED WHISKIES,260,25604,Seagrams 7 Crown Bl Whiskey,375,$3.50,$5.25,2,$10.50,0.75,0.20
366819,01/28/2016,4170,WEST BRANCH,52358,16.0,Cedar,1062200.0,PUERTO RICO & VIRGIN ISLANDS RUM,35,43127,Bacardi Superior Rum,1000,$9.50,$14.25,2,$28.50,2.00,0.53
366849,01/28/2016,4170,WEST BRANCH,52358,16.0,Cedar,1031080.0,VODKA 80 PROOF,260,37416,Popov Vodka 80 Prf,750,$4.50,$6.75,1,$6.75,0.75,0.20
366901,01/28/2016,4170,WEST BRANCH,52358,16.0,Cedar,1031080.0,VODKA 80 PROOF,260,37417,Popov Vodka 80 Prf,1000,$5.50,$8.25,1,$8.25,1.00,0.26


In [125]:
df_head[df_head['County'].isnull()]

,Date,Store_Number,City,Zip_Code,County_Number,County,Category,Category_Name,Vendor_Number,Item_Number,Item_Description,Bottle_Volume_(ml),State_Bottle_Cost,State_Bottle_Retail,Bottles_Sold,Sale_(Dollars),Volume_Sold_(Liters),Volume_Sold_(Gallons)
4,03/31/2016,5240,WEST BRANCH,52358,NaN,NaN,1081200.0,CREAM LIQUEURS,305,73055,Rumchata,750,$12.50,$18.75,6,$112.50,4.50,1.19
114,03/31/2016,5240,WEST BRANCH,52358,NaN,NaN,1011300.0,TENNESSEE WHISKIES,85,86670,Jack Daniel's Tennessee Honey,750,$15.07,$22.61,3,$67.83,2.25,0.59
209,03/31/2016,5240,WEST BRANCH,52358,NaN,NaN,1081400.0,TRIPLE SEC,65,82606,Dekuyper Sour Apple Pucker,750,$6.30,$9.45,3,$28.35,2.25,0.59
271,03/31/2016,5240,WEST BRANCH,52358,NaN,NaN,1081200.0,CREAM LIQUEURS,260,68034,Bailey's Original Irish Cream,375,$7.00,$10.50,3,$31.50,1.12,0.30
288,03/31/2016,5240,WEST BRANCH,52358,NaN,NaN,1081600.0,WHISKEY LIQUEUR,421,64864,Fireball Cinnamon Whisky,375,$5.33,$8.00,6,$48.00,2.25,0.59
290,03/31/2016,5240,WEST BRANCH,52358,NaN,NaN,1081300.0,PEPPERMINT SCHNAPPS,434,81196,Paramount Peppermint Schnapps Traveler,750,$3.75,$5.63,3,$16.89,2.25,0.59
296,03/31/2016,5240,WEST BRANCH,52358,NaN,NaN,1022100.0,TEQUILA,395,87408,Jose Cuervo Especial Silver,750,$10.49,$15.74,6,$94.44,4.50,1.19
323,03/31/2016,5240,WEST BRANCH,52358,NaN,NaN,1062310.0,SPICED RUM,240,45886,Sailor Jerry Spiced Navy Rum,750,$9.65,$14.48,3,$43.44,2.25,0.59
376,03/31/2016,5240,WEST BRANCH,52358,NaN,NaN,1051010.0,AMERICAN GRAPE BRANDIES,259,52316,Christian Bros Brandy,750,$5.92,$8.88,3,$26.64,2.25,0.59
379,03/31/2016,5240,WEST BRANCH,52358,NaN,NaN,1062200.0,PUERTO RICO & VIRGIN ISLANDS RUM,434,45275,Paramount White Rum Traveler,750,$4.00,$6.00,6,$36.00,4.50,1.19


In [72]:
# dfc.groupby("City")

# dfc[dfc['City'] == "DAVENPORT"]['County_Number'].value_counts()
# dfc[dfc['City'] == "BELMOND"]['County_Number'].value_counts()

for group_label, group_df in dfc.groupby(["City", "Store_Number"]):

    if len(group_df['County_Number'].value_counts()) > 1:
       print group_label, "count of subset: ", len(group_df.values)
       print "Number of unqiue county subsets: ", len(group_df['County_Number'].value_counts())

print "Unique cities: %s Unique Counties: %s" % (len(dfc['City'].value_counts()), len(dfc['County_Number'].value_counts()))

Unique cities: 386 Unique Counties: 99


In [ ]:
Iowa_Liqour_Sales.info(null_counts=True)

In [ ]:
from pandas_summary import DataFrameSummary

dfcs = DataFrameSummary(dfc)

In [ ]:
dfcs.summary().T

In [ ]:
# dfcs.summary()
dfcs.columns_stats.T

In [ ]:
dfc.info(null_counts=True)

In [ ]:
# Remove $ from certain columns
cols = ["State_Bottle_Cost", "State_Bottle_Retail", "Sale_Dollars"]
for col in cols:
    Iowa_Liqour_Sales[col] = Iowa_Liqour_Sales[col].apply(lambda x: float(x[1:]))

In [ ]:
# Convert dates
Iowa_Liqour_Sales["Date"] = pd.to_datetime(Iowa_Liqour_Sales["Date"], format="%m/%d/%Y")

In [ ]:
Iowa_Liqour_Sales['Year'] = Iowa_Liqour_Sales.Date.dt.year
Iowa_Liqour_Sales['Month'] = Iowa_Liqour_Sales.Date.dt.month

In [ ]:
#Move 'Year' and 'Month' columns after 'Date'

#Get a list of columns
cols = list(Iowa_Liqour_Sales)
cols.insert(1, cols.pop(cols.index('Year')))
cols.insert(2, cols.pop(cols.index('Month')))

#Use ix to reorder

Iowa_Liqour_Sales = Iowa_Liqour_Sales.ix[:,cols]
Iowa_Liqour_Sales.head(0)

In [ ]:
# Drop bad values
Iowa_Liqour_Sales = Iowa_Liqour_Sales.dropna()

In [ ]:
Iowa_Liqour_Sales.isnull().any()

In [ ]:
# Convert County Number and Category to integers
Iowa_Liqour_Sales["County_Number"] = Iowa_Liqour_Sales["County_Number"].astype(int)
Iowa_Liqour_Sales["Category"] = Iowa_Liqour_Sales["Category"].astype(int)
Iowa_Liqour_Sales.info()

In [132]:
Iowa_Liqour_Sales.isnull().sum()

Date                         0
Store_Number                 0
City                         0
Zip_Code                     0
County_Number            10500
County                   10500
Category                   779
Category_Name             6108
Vendor_Number                0
Item_Number                  0
Item_Description             0
Bottle_Volume_(ml)           0
State_Bottle_Cost            0
State_Bottle_Retail          0
Bottles_Sold                 0
Sale_(Dollars)               0
Volume_Sold_(Liters)         0
Volume_Sold_(Gallons)        0
dtype: int64

## Filter the Data

Some stores may have opened or closed in 2015. These data points will heavily skew our models, so we need to filter them out or find a way to deal with them.

You'll need to provide a summary in your project report about these data points. You may also consider using the monthly sales in your model and including other information (number of months or days each store is open) in your data to handle these unusual cases.

Let's record the first and last sales dates for each store. We'll save this information for later when we fit our models.